In [1]:
import torch
from pprint import pprint

print("Debugging...")
def custom_repr(self):
    return f'{{Tensor:{tuple(self.shape)}}}'
    # return f'{{Tensor:{tuple(self.shape)}}} {original_repr(self)}'

original_repr = torch.Tensor.__repr__
torch.Tensor.__repr__ = custom_repr

Debugging...


## Dataset

In [2]:
from main_trajectory import traj_collate_fn, fig_to_numpy, Arguments
from datasets.dataset_calvin import CalvinDataset
import pickle

pybullet build time: Nov 28 2023 23:51:11


In [3]:
# Parameters from the provided configuration
dense_interpolation = 1
interpolation_length = 20
relative_action = 1

image_rescale = "0.75,1.25"

In [4]:
def load_instructions(instructions, split):
    instructions = pickle.load(
        open(f"{instructions}/{split}.pkl", "rb")
    )['embeddings']
    return instructions

# Instruction paths
train_instruction = load_instructions(
    "../instructions/calvin_task_debug_D", 'training'
)
test_instruction = load_instructions(
    "../instructions/calvin_task_debug_D", 'validation'
)

taskvar = [
    ("A", 0), ("B", 0), ("C", 0), ("D", 0),
]

# Initialize datasets with specified arguments
def get_train_dataset():
    return CalvinDataset(
        root="../data/calvin/packaged_debug/training",
        instructions=train_instruction,
        taskvar=taskvar,
        max_episode_length=30,
        cache_size=0,
        max_episodes_per_task=-1,
        num_iters=600000,
        cameras=["front", "wrist"],
        training=True,
        image_rescale=tuple(float(x) for x in image_rescale.split(",")),
        return_low_lvl_trajectory=True,
        dense_interpolation=bool(dense_interpolation),
        interpolation_length=interpolation_length,
        relative_action=bool(relative_action)
    )
train_dataset = get_train_dataset()

print("Train dataset initialized")


Can't find dataset folder ../data/calvin/packaged_debug/training/A+0
Can't find dataset folder ../data/calvin/packaged_debug/training/B+0
Can't find dataset folder ../data/calvin/packaged_debug/training/C+0
Created dataset from ../data/calvin/packaged_debug/training with 9
Train dataset initialized


In [5]:
for episode in train_dataset:
    print(episode.keys())
    # vis sometime later, for now:
    print(episode)
    # 10 is timestep, frames in the episode
    # - 2 is multiview cam, 3 is rgb, 160x160
    # - 8 is [x, y, z], [convert_rotation (quat, but dunno xyzw or wxyz)], [open]
    # - - but calvin expects [x, y, z], [euler_x, euler_y, euler_z], [open]
    # - 16 is, uhh dunno, but 512 is the size of the embedding
    # - gripper is prolly 7 joint + gripper open/close?? doesnt make sense oherwise
    # - traj is next max 20 actions for the timestep

    # {
    #     'task': [
    #         'D', 'D', 'D', 'D', 'D',
    #         'D', 'D', 'D', 'D', 'D'
    #     ],
    #     'rgbs': {Tensor: (10, 2, 3, 160, 160)},
    #     'pcds': {Tensor: (10, 2, 3, 160, 160)},
    #     'action': {Tensor: (10, 8)},
    #     'instr': {Tensor: (10, 16, 512)},
    #     'curr_gripper': {Tensor: (10, 8)},
    #     'curr_gripper_history': {Tensor: (10, 3, 8)},
    #     'trajectory': {Tensor: (10, 20, 8)},
    #     'trajectory_mask': {Tensor: (10, 20)}
    # }
    break

dict_keys(['task', 'rgbs', 'pcds', 'action', 'instr', 'curr_gripper', 'curr_gripper_history', 'annotation_id', 'trajectory', 'trajectory_mask'])
{'task': ['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D'], 'rgbs': {Tensor:(10, 2, 3, 160, 160)}, 'pcds': {Tensor:(10, 2, 3, 160, 160)}, 'action': {Tensor:(10, 8)}, 'instr': {Tensor:(10, 16, 512)}, 'curr_gripper': {Tensor:(10, 8)}, 'curr_gripper_history': {Tensor:(10, 3, 8)}, 'annotation_id': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'trajectory': {Tensor:(10, 20, 8)}, 'trajectory_mask': {Tensor:(10, 20)}}


segregate 2d, 3d?

## Dataloader, collation

https://medium.com/geekculture/pytorch-datasets-dataloader-samplers-and-the-collat-fn-bbfc7c527cf1

In [6]:
from torch.utils.data import default_collate

In [7]:
default_collate(([1,2,3], [4,5,6]))

[{Tensor:(2,)}, {Tensor:(2,)}, {Tensor:(2,)}]

In [8]:
default_collate([1,2,3])

{Tensor:(3,)}

In [9]:
default_collate([[1,2,3], [4,5,6]])

[{Tensor:(2,)}, {Tensor:(2,)}, {Tensor:(2,)}]

In [10]:
default_collate([(1,2,3), (4,5,6)])

[{Tensor:(2,)}, {Tensor:(2,)}, {Tensor:(2,)}]

In [11]:
from main_trajectory import traj_collate_fn
# def traj_collate_fn(batch):
#     keys = [
#         "trajectory", "trajectory_mask",
#         "rgbs", "pcds",
#         "curr_gripper", "curr_gripper_history", "action", "instr"
#     ]
#     ret_dict = {
#         key: torch.cat([
#             item[key].float() if key != 'trajectory_mask' else item[key]
#             for item in batch
#         ]) for key in keys
#     }

#     ret_dict["task"] = []
#     ret_dict["annotation_id"] = []
#     for item in batch:
#         ret_dict["task"] += item['task']
#         ret_dict["annotation_id"] += item['annotation_id']
#     return ret_dict
collate_fn = traj_collate_fn

In [12]:
from torch.utils.data import DataLoader, default_collate
from torch.utils.data.distributed import DistributedSampler
import random
import numpy as np

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    np.random.seed(np.random.get_state()[1][0] + worker_id)

g = torch.Generator()
g.manual_seed(0)
# train_sampler = DistributedSampler(train_dataset)
train_loader = DataLoader(
    get_train_dataset(),
    batch_size=3,
    shuffle=False,
    num_workers=0, # it's 4 in the code, but lets run this in the main thread for debugging the collate_fn
    worker_init_fn=seed_worker,
    collate_fn=collate_fn,
    pin_memory=True,
    # sampler=train_sampler, # dont care about sampling really
    drop_last=True,
    generator=g
)
iter_loader = iter(train_loader)

Can't find dataset folder ../data/calvin/packaged_debug/training/A+0
Can't find dataset folder ../data/calvin/packaged_debug/training/B+0
Can't find dataset folder ../data/calvin/packaged_debug/training/C+0
Created dataset from ../data/calvin/packaged_debug/training with 9


In [13]:
batch = next(iter_loader)
print(batch.keys())
pprint(batch)

dict_keys(['trajectory', 'trajectory_mask', 'rgbs', 'pcds', 'curr_gripper', 'curr_gripper_history', 'action', 'instr', 'task', 'annotation_id'])
{'action': {Tensor:(24, 8)},
 'annotation_id': {Tensor:(24,)},
 'curr_gripper': {Tensor:(24, 8)},
 'curr_gripper_history': {Tensor:(24, 3, 8)},
 'instr': {Tensor:(24, 16, 512)},
 'pcds': {Tensor:(24, 2, 3, 160, 160)},
 'rgbs': {Tensor:(24, 2, 3, 160, 160)},
 'task': ['D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D',
          'D'],
 'trajectory': {Tensor:(24, 20, 8)},
 'trajectory_mask': {Tensor:(24, 20)}}


In [ ]:
import einops
from notebooks.visualizer import visualize_pointcloud
import torch

make2d = 2

def twoderp(pcds):
    pcds = pcds.clone()
    pcds[0, [2]] = 1
    pcds[1, [2]] = 0
    pcds[:, [0]], pcds[:, [1]] = torch.meshgrid(
        torch.linspace(-1, 1, pcds.shape[-2]),
        torch.linspace(1, -1, pcds.shape[-1]),
        indexing='xy'
    )
    return pcds

colors = batch['rgbs'][0] * 255

if make2d == 1:
    pcds = twoderp(batch['pcds'][0])
elif make2d == 2:
    pcds = batch['pcds'][0]
    tdpcds = twoderp(pcds)
    pcds = torch.cat([pcds, tdpcds], dim=0)
    colors = torch.cat([colors, colors], dim=0)
else:
    pcds = batch['pcds'][0]

visualize_pointcloud(einops.rearrange([pcds, colors], 'p n c h w -> (n h w) (p c)'))

 * Serving Flask app 'notebooks.visualizer'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Oct/2024 22:49:08] "GET / HTTP/1.1" 200 -
